# Arquitecturas para tareas de recomendación

In [ ]:
# Importación de librerías necesarias
import pandas as pd
import numpy as np
import sys
import numpy.core.numeric as _num
sys.modules['numpy._core.numeric'] = _num  # Solución a problemas de importación con ciertas versiones de numpy en algunos entornos (como Kaggle)

from sklearn.metrics import mean_squared_error  # Métrica para calcular el error cuadrático medio (usado para RMSE)

## Carga de los datasets



In [ ]:
# Se cargan los archivos preprocesados en formato pickle
train_obj = pd.read_pickle("/kaggle/input/ny8010/train_v80.pkl")
val_obj   = pd.read_pickle("/kaggle/input/ny8010/val_v10.pkl")
test_obj  = pd.read_pickle("/kaggle/input/ny8010/test_v10.pkl")

# Si por alguna razón los objetos son listas, se convierten a DataFrame
train_df = pd.DataFrame(train_obj) if isinstance(train_obj, list) else train_obj
val_df   = pd.DataFrame(val_obj) if isinstance(val_obj, list) else val_obj
test_df  = pd.DataFrame(test_obj) if isinstance(test_obj, list) else test_obj

# Mostrar el número de ejemplos disponibles en cada partición
print("Número de datos")
print("\nTrain:", len(train_df))
print("Val:", len(val_df))
print("Test:", len(test_df))
print("\nTotal:", len(train_df) + len(val_df) + len(test_df))

# Mostrar las columnas que contiene el DataFrame de entrenamiento
print("\nColumnas")
print(train_df.columns)

# Asignación de alias más cortos para los datasets
train = train_df
val = val_df
test = test_df

##  Evaluación de baseline

In [ ]:
# Asegurar que la columna 'rating' esté en formato float en los tres conjuntos
for df in (train, val, test):
    df['rating'] = df['rating'].astype(float)

# Definición de la función para calcular el RMSE
def rmse(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))

# Baseline: utilizar como predicción la media global de los ratings del conjunto de entrenamiento
global_mean = train['rating'].mean()

# Evaluación del baseline: se calcula el RMSE en train, val y test usando siempre la misma media
results = []
for name, df in [("Train", train), ("Val", val), ("Test", test)]:
    preds_global = np.full(len(df), global_mean)  # Array del mismo tamaño con la media como predicción constante
    results.append({
        'Baseline': 'Global Mean',
        'Dataset':  name,
        'RMSE':     rmse(df['rating'], preds_global)
    })

# Mostrar resultados en forma de tabla (filas = baseline, columnas = datasets)
df_results = pd.DataFrame(results)
print("\nRMSE del baseline (media global):")
print(df_results.pivot(index='Baseline', columns='Dataset', values='RMSE').round(4))


# Configuración inicial

In [ ]:
# Instalación y carga de librerías necesarias
import time                      # Para medir tiempos de ejecución
import warnings                  # Para controlar la visualización de advertencias
import torch                     # Framework principal de deep learning
import torch.nn as nn            # Módulo para redes neuronales
import torch.optim as optim      # Optimizadores como Adam
import pandas as pd              # Carga y manejo de datos
import numpy as np               # Operaciones numéricas con arrays
import matplotlib.pyplot as plt  # Visualización de resultados
from torch.utils.data import Dataset, DataLoader  # Dataset y cargadores en PyTorch
from tqdm.auto import tqdm       # Barra de progreso para bucles

# Configuración para ignorar warnings de tipo FutureWarning
warnings.filterwarnings("ignore", category=FutureWarning)

# Configura el dispositivo: usa GPU si está disponible, sino CPU
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

# Habilita el uso de precisión mixta automática si hay GPU (más rápido y eficiente)
use_amp = DEVICE == "cuda"

# Definición de hiperparámetros para el entrenamiento de todos los modelos
BATCH_SIZE = 128          # Tamaño de batch para los DataLoaders
MAX_EPOCHS = 1000         # Número máximo de épocas de entrenamiento
PATIENCE = 10             # Número de épocas sin mejora para activar early stopping
EMB_DIM = 50              # Dimensión de los embeddings de usuario e ítem
HIDDEN_DIM = 64           # Tamaño de la capa oculta en redes MLP
LR_LIST = [1e-5, 1e-4, 5e-4, 1e-3]  # Lista de tasas de aprendizaje a probar
WEIGHT_DECAY = 1e-6       # Regularización para evitar overfitting (L2)


# clases dataset

Las clases Dataset personalizadas definen cómo se organizan y extraen los datos desde un DataFrame para adaptarlos al entrenamiento de modelos en PyTorch. Cada clase está diseñada para un tipo específico de entrada: algunas usan identificadores numéricos de usuario y restaurante (FM_Dataset), otras trabajan con embeddings de texto o imagen generados previamente (CLIPTextDataset, CLIPImageDataset), y otras combinan distintas fuentes de información (MixedDataset, FullDataset). Estas clases son necesarias para que PyTorch pueda acceder a los datos de forma estructurada y eficiente.

A partir de estas clases, se crean objetos llamados DataLoaders, que se encargan de cargar los datos por lotes (batches), barajarlos si es necesario, y alimentar automáticamente al modelo durante el entrenamiento y validación. Esto permite aprovechar mejor la GPU, reducir el consumo de memoria y facilitar el entrenamiento con datasets grandes. En resumen, Dataset define cómo se obtienen los datos, y DataLoader gestiona cómo se entregan al modelo en cada iteración.

In [ ]:

# ============================================
# Definición de clases Dataset personalizadas
# ============================================

# Dataset clásico basado en codificación de usuario e ítem con sus IDs
class FM_Dataset(Dataset):
    def __init__(self, df):
        # Convierte las columnas de IDs y rating a tensores de PyTorch
        self.u = torch.tensor(df['user_id_new'].values, dtype=torch.long)
        self.i = torch.tensor(df['restaurant_id_new'].values, dtype=torch.long)
        self.r = torch.tensor(df['rating'].values, dtype=torch.float32)
    def __len__(self): return len(self.r)
    def __getitem__(self, idx): return self.u[idx], self.i[idx], self.r[idx]

# Dataset para modelos que usan embeddings CLIP de texto como entrada
class CLIPTextDataset(Dataset):
    def __init__(self, df):
        # Combina los vectores de texto en una matriz y convierte a tensor
        self.x = torch.from_numpy(np.vstack(df['text_emb'].values)).float()
        self.y = torch.tensor(df['rating'].values, dtype=torch.float32)
    def __len__(self): return len(self.y)
    def __getitem__(self, idx): return self.x[idx], self.y[idx]

# Dataset mixto que combina IDs con embeddings CLIP de texto
class MixedDataset(Dataset):
    def __init__(self, df):
        self.u = torch.tensor(df['user_id_new'].values, dtype=torch.long)
        self.i = torch.tensor(df['restaurant_id_new'].values, dtype=torch.long)
        self.x = torch.from_numpy(np.vstack(df['text_emb'].values)).float()
        self.r = torch.tensor(df['rating'].values, dtype=torch.float32)
    def __len__(self): return len(self.r)
    def __getitem__(self, idx): return self.u[idx], self.i[idx], self.x[idx], self.r[idx]

# Dataset para modelos que usan solo embeddings CLIP de imagen como entrada
class CLIPImageDataset(Dataset):
    def __init__(self, df):
        self.x = torch.from_numpy(np.stack(df['image_emb'].values)).float()
        self.y = torch.tensor(df['rating'].astype(float).values, dtype=torch.float32)
    def __len__(self): return len(self.y)
    def __getitem__(self, idx): return self.x[idx], self.y[idx]

# Dataset para modelos que combinan embeddings CLIP de imagen y texto
class CLIPMixedDataset(Dataset):
    def __init__(self, df):
        # Concatena los embeddings de imagen y texto horizontalmente (por columnas)
        img_emb = np.stack(df['image_emb'].values)
        txt_emb = np.stack(df['text_emb'].values)
        self.x = torch.from_numpy(np.concatenate([img_emb, txt_emb], axis=1)).float()
        self.y = torch.tensor(df['rating'].astype(float).values, dtype=torch.float32)
    def __len__(self): return len(self.y)
    def __getitem__(self, idx): return self.x[idx], self.y[idx]

# Dataset más completo: incluye IDs, embeddings de imagen y de texto
class FullDataset(Dataset):
    def __init__(self, df):
        self.u = torch.tensor(df['user_enc'].values, dtype=torch.long)
        self.i = torch.tensor(df['item_enc'].values, dtype=torch.long)
        img_emb = np.stack(df['image_emb'].values)
        txt_emb = np.stack(df['text_emb'].values)
        self.x = torch.from_numpy(np.concatenate([img_emb, txt_emb], axis=1)).float()
        self.y = torch.tensor(df['rating'].astype(float).values, dtype=torch.float32)
    def __len__(self): return len(self.y)
    def __getitem__(self, idx): return self.u[idx], self.i[idx], self.x[idx], self.y[idx]


# Aquitecutras

## Tabla resumen de arquitecturas de recomendación

| Nº | Nombre del Modelo           | Entradas utilizadas                                  | Tipo de arquitectura         | Comentario breve |
|----|-----------------------------|------------------------------------------------------|------------------------------|------------------|
| 1  | MatrixFactorization         | ID de usuario + ID de restaurante                    | Producto punto               | Factorización clásica sin red neuronal |
| 2  | NeuralRecommender           | ID de usuario + ID de restaurante                    | MLP                          | Recomendador neuronal que concatena embeddings |
| 3  | CLIPTextRegressor           | Embedding CLIP del texto                             | MLP                          | Solo utiliza el contenido textual |
| 4  | MixedModel                  | ID de usuario + ID de restaurante + texto emb       | MLP                          | Mezcla codificación tradicional con texto |
| 5  | CLIPImageRegressor          | Embedding CLIP de imagen                             | MLP                          | Solo utiliza el contenido visual |
| 6  | CLIPMixedRegressor          | Embedding CLIP de imagen + texto                    | MLP                          | Fusiona ambos tipos de contenido multimodal |
| 7  | FullModel                   | ID de usuario + ID de restaurante + img + texto     | MLP                          | Modelo más completo, combina IDs y multimodalidad |


In [ ]:
# ============================================
# Definición de modelos de recomendación
# ============================================

# Arquitectura 1: Factorización Matricial
# Aprende un vector de características (embedding) por usuario y por ítem.
# La predicción del rating se obtiene como el producto punto entre ambos embeddings.
class MatrixFactorization(nn.Module):
    def __init__(self, n_users, n_items):
        super().__init__()
        # Embeddings para usuarios e ítems
        self.Eu = nn.Embedding(n_users, EMB_DIM)
        self.Ei = nn.Embedding(n_items, EMB_DIM)
        # Inicialización normal de pesos
        nn.init.normal_(self.Eu.weight, mean=1.0, std=0.01)
        nn.init.normal_(self.Ei.weight, mean=1.0, std=0.01)

    def forward(self, u, i):
        # Producto punto entre los embeddings del usuario y del ítem
        return (self.Eu(u) * self.Ei(i)).sum(dim=1)

# Arquitectura 2: Recomendador neuronal (MLP)
# Similar a la anterior, pero en vez de usar producto punto, combina los embeddings con una red neuronal.
class NeuralRecommender(nn.Module):
    def __init__(self, n_users, n_items):
        super().__init__()
        # Embeddings para usuarios e ítems
        self.user_emb = nn.Embedding(n_users, EMB_DIM)
        self.item_emb = nn.Embedding(n_items, EMB_DIM)
        # Inicialización normal de pesos
        nn.init.normal_(self.user_emb.weight, mean=1.0, std=0.01)
        nn.init.normal_(self.item_emb.weight, mean=1.0, std=0.01)
        # Red neuronal para predecir el rating a partir de los embeddings concatenados
        self.mlp = nn.Sequential(
            nn.Linear(2 * EMB_DIM, HIDDEN_DIM),
            nn.ReLU(),
            nn.Linear(HIDDEN_DIM, 1)
        )

    def forward(self, u, i):
        # Concatenación de los embeddings de usuario e ítem
        x = torch.cat([self.user_emb(u), self.item_emb(i)], dim=1)
        # Predicción del rating
        return self.mlp(x).squeeze(1)

# Arquitecturas 3, 5 y 6: Regresores para embeddings (texto, imagen o combinación)
# Red neuronal que toma directamente un vector (embedding CLIP) como entrada y predice un rating.
class CLIPRegressor(nn.Module):
    def __init__(self, input_dim):
        super().__init__()
        # Red MLP con 3 capas para regresión
        self.mlp = nn.Sequential(
            nn.Linear(input_dim, 100),
            nn.ReLU(),
            nn.Linear(100, 64),
            nn.ReLU(),
            nn.Linear(64, 1)
        )

    def forward(self, x):
        return self.mlp(x).squeeze(1)

# Arquitectura 4: Modelo mixto (IDs + embeddings de texto)
# Combina los embeddings de usuario e ítem con los vectores CLIP de texto.
class MixedModel(nn.Module):
    def __init__(self, n_users, n_items, clip_dim):
        super().__init__()
        self.user_emb = nn.Embedding(n_users, EMB_DIM)
        self.item_emb = nn.Embedding(n_items, EMB_DIM)
        nn.init.normal_(self.user_emb.weight, mean=1.0, std=0.01)
        nn.init.normal_(self.item_emb.weight, mean=1.0, std=0.01)
        # MLP que procesa los embeddings junto con el texto
        self.mlp = nn.Sequential(
            nn.Linear(clip_dim + 2 * EMB_DIM, 100),
            nn.ReLU(),
            nn.Linear(100, 64),
            nn.ReLU(),
            nn.Linear(64, 1)
        )

    def forward(self, u, i, x):
        # Concatenación de embeddings de usuario, ítem y texto CLIP
        x = torch.cat([self.user_emb(u), self.item_emb(i), x], dim=1)
        return self.mlp(x).squeeze(1)

# Arquitectura 7: Modelo completo (IDs + imagen + texto)
# Utiliza toda la información disponible: IDs, embeddings CLIP de imagen y de texto.
class FullModel(nn.Module):
    def __init__(self, n_users, n_items, clip_dim=1024, emb_dim=50):
        super().__init__()
        self.user_emb = nn.Embedding(n_users, emb_dim)
        self.item_emb = nn.Embedding(n_items, emb_dim)
        nn.init.normal_(self.user_emb.weight, mean=1.0, std=0.01)
        nn.init.normal_(self.item_emb.weight, mean=1.0, std=0.01)
        # MLP que combina embeddings de usuario, ítem y embeddings CLIP (texto + imagen)
        self.mlp = nn.Sequential(
            nn.Linear(clip_dim + 2 * emb_dim, 100),
            nn.ReLU(),
            nn.Linear(100, 64),
            nn.ReLU(),
            nn.Linear(64, 1)
        )

    def forward(self, u, i, x):
        # Concatenación de toda la información (IDs + texto + imagen)
        concat = torch.cat([self.user_emb(u), self.item_emb(i), x], dim=1)
        return self.mlp(concat).squeeze(1)


# Entrenar y resultados

In [ ]:
# ============================================
# Funciones auxiliares: entrenamiento y evaluación
# ============================================

# Función para calcular el RMSE en un conjunto de datos (validación o test)
def compute_rmse(model, loader, criterion):
    model.eval()  # Se pone el modelo en modo evaluación (desactiva dropout, batchnorm, etc.)
    total, n = 0.0, 0
    with torch.no_grad():  # No se calculan gradientes
        for batch in loader:
            batch = [b.to(DEVICE) for b in batch]  # Se envían los tensores al dispositivo
            pred, y = forward_batch(model, batch)  # Se obtienen predicciones y etiquetas reales
            loss = criterion(pred, y)  # Se calcula el error cuadrático medio
            total += loss.item() * y.size(0)  # Se acumula el error total
            n += y.size(0)  # Se acumulan las muestras procesadas
    return np.sqrt(total / n)  # Se devuelve la raíz del error cuadrático medio (RMSE)

# Función generalizada para hacer forward pass compatible con distintos tipos de entrada
def forward_batch(model, batch):
    if len(batch) == 2:
        # Casos con solo features (x) y etiquetas (y)
        x, y = batch
        pred = model(x)
    elif len(batch) == 3:
        # Casos con user ID, item ID y etiquetas (factorización matricial o MLP con IDs)
        u, i, y = batch
        pred = model(u, i)
    else:
        # Casos con user ID, item ID, embeddings (texto/imagen) y etiquetas
        u, i, x, y = batch
        pred = model(u, i, x)
    return pred, y

# Función principal para entrenar un modelo con validación y early stopping
def train_model(name, model_class, dataset_class, train_df, val_df, test_df, extra_args={}):
    print(f"\n===> Entrenando {name}")

    # Creación de DataLoaders para entrenamiento, validación y test
    train_loader = DataLoader(dataset_class(train_df), batch_size=BATCH_SIZE, shuffle=True)
    val_loader   = DataLoader(dataset_class(val_df), batch_size=BATCH_SIZE)
    test_loader  = DataLoader(dataset_class(test_df), batch_size=BATCH_SIZE)

    criterion = nn.MSELoss()  # Se utiliza el error cuadrático medio como función de pérdida
    results = []  # Lista para almacenar resultados por tasa de aprendizaje

    # Se entrena el modelo con distintas tasas de aprendizaje
    for lr in LR_LIST:
        model = model_class(**extra_args).to(DEVICE)  # Se inicializa el modelo
        optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=WEIGHT_DECAY)  # Optimizador Adam
        scaler = torch.cuda.amp.GradScaler() if use_amp else None  # Escalado automático de precisión (AMP)

        # Variables para early stopping
        best_val, best_state, best_epoch = float('inf'), None, 0
        train_hist, val_hist = [], []
        no_improve = 0

        # Entrenamiento por épocas
        for epoch in tqdm(range(1, MAX_EPOCHS + 1), desc=f"{name} lr={lr}"):
            model.train()
            total_loss, count = 0.0, 0

            for batch in train_loader:
                batch = [b.to(DEVICE) for b in batch]
                optimizer.zero_grad()

                if use_amp:
                    # Entrenamiento con AMP (si se usa)
                    with torch.cuda.amp.autocast():
                        pred, y = forward_batch(model, batch)
                        loss = criterion(pred, y)
                    scaler.scale(loss).backward()
                    scaler.step(optimizer)
                    scaler.update()
                else:
                    # Entrenamiento clásico sin AMP
                    pred, y = forward_batch(model, batch)
                    loss = criterion(pred, y)
                    loss.backward()
                    optimizer.step()

                total_loss += loss.item() * y.size(0)
                count += y.size(0)

            # Evaluación tras cada época
            train_rmse = np.sqrt(total_loss / count)
            val_rmse = compute_rmse(model, val_loader, criterion)
            train_hist.append(train_rmse)
            val_hist.append(val_rmse)

            # Verificación para early stopping
            if val_rmse < best_val:
                best_val, best_state, best_epoch = val_rmse, model.state_dict(), epoch
                no_improve = 0
            else:
                no_improve += 1
                if no_improve >= PATIENCE:  # No mejora tras 'PATIENCE' épocas
                    break

        # Se carga el mejor modelo y se evalúa en test
        model.load_state_dict(best_state)
        test_rmse = compute_rmse(model, test_loader, criterion)

        # Se guarda el resultado para esta tasa de aprendizaje
        results.append({
            'Arquitectura': name,
            'learning_rate': lr,
            'train_rmse': train_hist[best_epoch - 1],
            'val_rmse': best_val,
            'test_rmse': test_rmse,
            'epochs': best_epoch
        })

        # Guardado de la curva de entrenamiento
        plt.figure()
        plt.plot(train_hist, label='Train RMSE')
        plt.plot(val_hist, label='Val RMSE')
        plt.axvline(best_epoch, linestyle='--', color='gray', label='Best Epoch')
        plt.xlabel('Epoch')
        plt.ylabel('RMSE')
        plt.title(f'{name} @ lr={lr}')
        plt.legend()
        plt.grid(True)
        plt.tight_layout()
        plt.savefig(f'curve_{name.replace("-", "_")}_lr_{lr}.pdf')
        plt.close()

    # Exportación de resultados a Excel
    df = pd.DataFrame(results)
    df.to_excel(f"{name}.xlsx", index=False)
    print(f"✅ Resultados guardados en {name}.xlsx")

# ============================================
# Lista de arquitecturas a entrenar
# ============================================

# Cada tupla incluye:
# - Nombre del modelo
# - Clase del modelo (PyTorch)
# - Dataset correspondiente
# - Parámetros adicionales requeridos (como nº de usuarios, ítems o dimensión de embeddings)
ARQUITECTURAS = [
    ("1-FM", MatrixFactorization, FM_Dataset, {'n_users': train['user_id_new'].nunique(), 'n_items': train['restaurant_id_new'].nunique()}),
    ("2-RN", NeuralRecommender, FM_Dataset, {'n_users': train['user_id_new'].nunique(), 'n_items': train['restaurant_id_new'].nunique()}),
    ("3-CLIP-TEXT", CLIPRegressor, CLIPTextDataset, {'input_dim': len(train['text_emb'].iloc[0])}),
    ("4-MIX-TXTEN", MixedModel, MixedDataset, {'n_users': train['user_id_new'].nunique(), 'n_items': train['restaurant_id_new'].nunique(), 'clip_dim': len(train['text_emb'].iloc[0])}),
    ("5-CLIP-IMG", CLIPRegressor, CLIPImageDataset, {'input_dim': len(train['image_emb'].iloc[0])}),
    ("6-TXT+IMG", CLIPRegressor, CLIPMixedDataset, {'input_dim': len(train['image_emb'].iloc[0]) + len(train['text_emb'].iloc[0])}),
    ("7-FULL", FullModel, FullDataset, {'n_users': train['user_enc'].nunique(), 'n_items': train['item_enc'].nunique()})
]

# Entrenamiento de todas las arquitecturas con evaluación y guardado
start_all = time.time()
for name, model_cls, dataset_cls, extra_args in ARQUITECTURAS:
    train_model(name, model_cls, dataset_cls, train, val, test, extra_args)
print(f"\n⏱️ Tiempo total: {(time.time() - start_all)/60:.2f} minutos")
